In [1]:
from sqlalchemy import create_engine
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from collections import defaultdict
from dotenv import load_dotenv

In [2]:
load_dotenv()

In [4]:
DB_PARAMS = {
    "dbname": os.getenv("POSTGRES_DB"),
    "user": os.getenv("POSTGRES_USER"),
    "password": os.getenv("POSTGRES_PASSWORD"), 
    "host": "postgres",
    "port":  5432
}

In [83]:
try:
    db_url = f"postgresql+psycopg2://{DB_PARAMS['user']}:{DB_PARAMS['password']}@{DB_PARAMS['host']}/{DB_PARAMS['dbname']}"
    engine = create_engine(db_url)

    query = """
        SELECT 
            user_id,
            COUNT(*) AS purchase_count,
            SUM(price::numeric) AS total_spending
        FROM customers
        WHERE event_type = 'purchase'
        GROUP BY user_id
        HAVING SUM(price::numeric) < 225
           AND COUNT(*) <= 40;
    """
    df = pd.read_sql(query, engine)

except Exception as e:
    print(f"❌ An error occurred: {e}")

In [105]:
frequency = df['purchase_count']
monetary = df['total_spending'] 

In [ ]:
sns.set_style("whitegrid")
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

axes[0].set_facecolor('lightgrey')
axes[1].set_facecolor('lightgrey')

sns.histplot(frequency, bins=5, kde=False, color='lightblue', ax=axes[0]) 
axes[0].set_ylabel('customers')
axes[0].set_xlabel('frequency')
axes[0].set_xticks(range(0, 39, 10))

axes[1].hist(monetary, bins=5)
axes[1].set_ylabel('customers')
axes[1].set_xlabel('Monetary value in Altairian Dollars')